<h2>FT004 - Cuentas por Pagar </h2>
<p> En este apartado se trata la base de datos FT004 </p>

In [ ]:
#Cargamos los paquetes que vamos a necesitar
import pandas as pd
import numpy as np
import pyreadstat
pd.pandas.set_option('display.max_columns', None)

In [ ]:
# Se carga el FT005 completo desde el archivo .dta
FT004_2021_2021 = pd.read_stata('C:/Users/Miguel Angel/Documents/Supersalud/Pagos_EPS_Proveedores/Informe_Pagos/FT004 20202021.dta')

In [ ]:
# Se carga el FT004 para diciembre de 2019 desde el archivo .dta
FT004_2019 = pd.read_stata('C:/Users/Miguel Angel/Documents/Supersalud/Pagos_EPS_Proveedores/Informe_Pagos/FT004 122019.dta')

In [ ]:
# Se concatenan la bases de datos 
frames = [FT004_2021_2021, FT004_2019]
FT004 = pd.concat(frames, ignore_index=True)

In [ ]:
# Se eliminan la bases que no se necesitan para liberar espacio en la memoria
del [FT004_2021_2021, FT004_2019, frames]

In [ ]:
# Se reemplazan los nombres de las columnas para facilitar el tratamiento
FT004.rename(columns={'Nit': 'nit_eps', 'Año': 'ano', 'Periodo': 'mes', 'lineaNegocio': 'linea','tipoIdAcreedor': 'tipoidacreedor', 'idAcreedor': 'nitproveedor', 'dvAcreedor': 'DVacreedor', 'conceptoAcreencia': 'concepto', 'cxpNoVencidas': 'cxpnovencidas', 'cxpMora30dias': 'cxp30','cxpMora60dias': 'cxp60', 'cxpMora90dias': 'cxp90', 'cxpMora180dias': 'cxp180', 'cxpMora360dias': 'cxp360','cxpMoraMayor360dias': 'cxpmayor360'}, inplace=True)

In [ ]:
# Se transforman los datos para su correcta identificación
FT004[['nit_eps', 'RazonSocial', 'ano', 'mes', 'linea', 'tipoidacreedor',
       'nitproveedor', 'DVacreedor', 'nombreAcreedor', 'actividadAcreedor',
       'concepto', 'medicionPosterior']] = FT004[['nit_eps', 'RazonSocial', 'ano', 'mes', 'linea', 'tipoidacreedor',
       'nitproveedor', 'DVacreedor', 'nombreAcreedor', 'actividadAcreedor',
       'concepto', 'medicionPosterior']].astype(str)

FT004[['cxpnovencidas', 'cxp30', 'cxp60',
       'cxp90', 'cxp180', 'cxp360', 'cxpmayor360', 'ajuste', 'saldo']] = FT004[['cxpnovencidas', 'cxp30', 'cxp60',
       'cxp90', 'cxp180', 'cxp360', 'cxpmayor360', 'ajuste', 'saldo']].astype(float)

In [ ]:
# Se filtra la línea de negocio de interés. En este caso corresponde a la línea 1
FT004 = FT004.loc[FT004['linea'] == '1']

In [ ]:
FT004.columns

In [ ]:
# Se cuenta el número de duplicados para hacer la respectiva corrección del tipo id deudor
df_acreedor = FT004.groupby(['nitproveedor', 'nombreAcreedor']).size().reset_index().rename(columns={0:'Conteo_FT04'})

In [ ]:
# Para guardar la base de datos lista para ser tratada en formato Excel y CSV
df_acreedor.to_excel(r'C:/Users/Miguel Angel/Documents/Supersalud/Pagos_EPS_Proveedores/df_acreedor_FT004_V2.xlsx', index = False)

In [ ]:
# Se crea la columna Cuentas por Pagar (CXP) sumando las columnas cxpnovencidas, cxp30, cxp60, cxp90, cxp180, cxp360 y cxpmayor360
CXP = FT004['cxpnovencidas'] + FT004['cxp30'] + FT004['cxp60'] + FT004['cxp90'] + FT004['cxp180'] + FT004['cxp360'] + FT004['cxpmayor360']
FT004['CXP'] = CXP

In [ ]:
# Para hacer la suma de CXC por NIT EPS, tipo de proveedor, NIT proveedor, Mes y Año
FT004 = FT004.groupby(['nit_eps', 'tipoidacreedor', 'nitproveedor', 'mes', 'ano']).agg({'CXP': 'sum'}).reset_index()

In [ ]:
# Se comprueba que no haya duplicados cuando se filtra por NIT EPS, tipo de deudor, NIT proveedor, Mes y Año. Para esto se cuenta el número de duplicados
duplicadosFT004 = FT004[FT004.duplicated(['nit_eps', 'tipoidacreedor', 'nitproveedor', 'mes', 'ano'], keep='last')]
print("El número de registros duplicados es:", duplicadosFT004.nit_eps.count(), sep='\n')

In [ ]:
# Se carga la base de datos cuenta2.dta para su posterior pega
cuentas2 = pd.read_stata('C:/Users/Miguel Angel/Documents/Supersalud/Pagos_EPS_Proveedores/cuentas2.dta')

In [ ]:
# Se tranforma el formato de las columnas del dataframe cuentas2
cuentas2[['v2', 'nitproveedor']] = cuentas2[['v2', 'nitproveedor']].astype(str)

In [ ]:
# Se comprueba que no haya duplicados cuando se filtra por Nitproveedor. Para esto se cuenta el número de duplicados
duplicados_cuentas2 = cuentas2[cuentas2.duplicated(['nitproveedor'], keep='last')]
print("El número de registros duplicados es:", duplicados_cuentas2.nitproveedor.count(), sep='\n')

In [ ]:
# Se unen las dos bases de datos para obtener la información de FT004 con cuenta2
FT004= pd.merge(FT004, cuentas2, how='left', left_on=['nitproveedor'], right_on=['nitproveedor'])

In [ ]:
<p> Esto corresponde a la cuenta del pasivo cuanso se agregan las cuentas por pagar. Por tal motivo se elimna ya que no se pueden identificar los proveedores. </p>

In [ ]:
# Se encuentran las observaciones que cumplen las condiciones
df_filtered = FT004[(FT004['tipoidacreedor'] == 'OT') & (FT004['v2'] == 'cuenta')].index

In [ ]:
# Se eliminan las observaciones que cumplen las condiciones anteriore
FT004 = FT004.drop(df_filtered)

In [ ]:
# Se genera una nueva columna extrayendo el primer caracter del ID nitproveedor
FT004['d1'] = FT004['nitproveedor'].str[:1]

In [ ]:
# Se genera una nueva columna con la cuenta del numero de componentes que tiene el ID nitproveedor
FT004['largo'] = FT004.nitproveedor.str.len()

In [ ]:
# Se limpia la clasificación por Tipo de acreedor
FT004['tipoidacreedor_corregido'] = FT004['tipoidacreedor'] # Se crea la columna con categoria NA como base
# Condiciones
FT004.loc[
       (FT004['tipoidacreedor_corregido'] == 'NA') &
       (FT004['largo'] == 10) & # Para signar el tipo de acreedor CC
       (FT004['d1'] == '1'), 
       'tipoidacreedor_corregido'] = 'CC' 
FT004.loc[(FT004['tipoidacreedor_corregido'] == 'NA') &
       (FT004['largo'] == 9) & # Para signar el tipo de acreedor NI
       ((FT004['d1'] == '7') |
       (FT004['d1'] == '8') |
       (FT004['d1'] == '9')),  
       'tipoidacreedor_corregido'] = 'NI' 
FT004.loc[(FT004['tipoidacreedor_corregido'] == 'NA') &
       (FT004['tipoidacreedor_corregido'] != 'NI'), # Para completar el tipo de acreedor CC
       'tipoidacreedor_corregido'] = 'CC' 

In [ ]:
# Para conocer las diferencias entre las correciones
print(FT004.tipoidacreedor.value_counts())
print(FT004.tipoidacreedor_corregido.value_counts())

In [ ]:
# Para hacer la suma de CXP por NIT EPS, tipo de acreedor corregido, NIT proveedor, Mes y Año
FT004 = FT004.groupby(['nit_eps', 'tipoidacreedor_corregido', 'nitproveedor', 'mes', 'ano']).agg({'CXP': 'sum'}).reset_index()

In [ ]:
# Se reemplazan los nombres de las columnas para facilitar el tratamiento
FT004.rename(columns={'tipoidacreedor_corregido': 'tipoidacreedor'}, inplace=True)

In [ ]:
FT004[['ano', 'mes']] = FT004[['ano', 'mes']].astype(int)

In [ ]:
# Se crea el rango de meses necesarios en cada mes
fecha_ideal=range(1, 12 + 1)
# Se rellena con ceros la información para los meses faltantes
FT004 = FT004.set_index('mes').groupby(['nit_eps', 'tipoidacreedor', 'nitproveedor', 'ano']).apply(lambda x: x.reindex(index=fecha_ideal, fill_value=0)).drop(['nit_eps', 'tipoidacreedor', 'nitproveedor', 'ano'], 1).reset_index()

In [ ]:
# Se encuentran las observaciones que cumplen las condiciones
df_filtered = FT004[
    (FT004['ano'] == 2019) & (FT004['mes'] == 1) |
    (FT004['ano'] == 2019) & (FT004['mes'] == 2) |
    (FT004['ano'] == 2019) & (FT004['mes'] == 3) |
    (FT004['ano'] == 2019) & (FT004['mes'] == 4) |
    (FT004['ano'] == 2019) & (FT004['mes'] == 5) |
    (FT004['ano'] == 2019) & (FT004['mes'] == 6) |
    (FT004['ano'] == 2019) & (FT004['mes'] == 7) |
    (FT004['ano'] == 2019) & (FT004['mes'] == 8) |
    (FT004['ano'] == 2019) & (FT004['mes'] == 9) |
    (FT004['ano'] == 2019) & (FT004['mes'] == 10) |
    (FT004['ano'] == 2019) & (FT004['mes'] == 11)
    ].index

# Se eliminan las observaciones que cumplen las condiciones anteriore
FT004 = FT004.drop(df_filtered)

In [ ]:
del [CXP, cuentas2, df_filtered, duplicados_cuentas2, duplicadosFT004, fecha_ideal]

In [ ]:
%%time
# Se completan los años y los meses para las observaciones que no presentaron ninguna observación en un año de interés. Por ejemplo, a aquellos sujetos que reportaron cifras en 2019, pero no en 2020, se les completa el año 2020 para poder calcular la diferencia entre diciembre de 2019 y enero de 2020
# Se convierte la variable en mes y año
FT004[['ano', 'mes']] = FT004[['ano', 'mes']].astype(float)
FT004['periodo'] = pd.to_datetime(FT004.ano*10000 + FT004.mes*100 + 1, format='%Y%m%d')
fecha_ideal = pd.date_range('12-01-2019','05-01-2021',freq='M')

In [ ]:
FT004_dif = FT004.groupby(['nit_eps', 'tipoidacreedor', 'nitproveedor', pd.Grouper(key='periodo', freq='M')])[['CXP']].sum().reset_index()

In [ ]:
FT004_dif = FT004_dif.set_index('periodo').groupby(['nit_eps', 'tipoidacreedor', 'nitproveedor']).apply(lambda x: x.reindex(index=fecha_ideal, fill_value=0)).drop(['nit_eps', 'tipoidacreedor', 'nitproveedor'], 1).reset_index()

In [ ]:
# Se reemplazan los nombres de las columnas para facilitar el tratamiento
FT004_dif.rename(columns={'level_3': 'periodo'}, inplace=True)

In [ ]:
FT004_dif = FT004_dif.set_index(['nit_eps', 'nitproveedor', 'tipoidacreedor', 'periodo'])

In [ ]:
%%time
# Se calcula la diferencia mensual para conocer el valor mensual de cada cuenta ya que por defecto vienen agregadas
FT004_dif = FT004_dif.diff().fillna(0).reset_index()

In [ ]:
# Se reemplazan los nombres de las columnas para facilitar el tratamiento
FT004_dif.rename(columns={'CXP': 'DCXPmes'}, inplace=True)

#  Se extrae el mes y el año para porterior left join con el agregado
FT004_dif['ano'] = FT004_dif['periodo'].dt.year
FT004_dif['mes'] = FT004_dif['periodo'].dt.month

#Se transforman las variables a STR para mejorar su manipulación
FT004[['ano', 'mes']] = FT004[['ano', 'mes']].astype(int)

# Se eliminan las columnas que no son de interés
FT004 = FT004.drop(['periodo'], axis=1)
FT004_dif = FT004_dif.drop(['periodo'], axis=1)

In [ ]:
# Se unen las dos bases de datos para obtener la información de webgironoupc (Giro Directo - No_UPC - Compra de Cartera) y FT005 en una sola base de datos. Desagregando el valor de VCausado y el VPagado para cada mes
FT004 = pd.merge(FT004_dif, FT004, how='left', left_on=['nit_eps', 'tipoidacreedor', 'nitproveedor', 'mes', 'ano'], right_on=['nit_eps', 'tipoidacreedor', 'nitproveedor', 'mes', 'ano'])

In [ ]:
# Se encuentran las observaciones que cumplen las condiciones
df_filtered = FT004[
    (FT004['ano'] == 2019)
    ].index

# Se eliminan las observaciones que cumplen las condiciones anteriore
FT004 = FT004.drop(df_filtered)

In [ ]:
# Se filtra el dataframe con las observaciones que cumplen las condiciones
FT004_V2 = FT004[
    (FT004['nitproveedor'] == '800015826')
    ]

In [ ]:
del [df_filtered, fecha_ideal]

In [ ]:
# Se carga la base de datos prestadores.dta para su posterior pega
prestadores = pd.read_stata('C:/Users/Miguel Angel/Documents/Supersalud/Pagos_EPS_Proveedores/prestadores.dta')

In [ ]:
# Se comprueba que no haya duplicados cuando se filtra por Nitproveedor. Para esto se cuenta el número de duplicados
duplicados_prestadores = prestadores[prestadores.duplicated(['nitproveedor'], keep='last')]
print("El número de registros duplicados es:", duplicados_prestadores.nitproveedor.count(), sep='\n')

In [ ]:
# Se unen las dos bases de datos para obtener la información de FT005 con webgironoupc (Giro Directo - No_UPC - Compra de Cartera) y prestadores
FT004 = pd.merge(FT004, prestadores, how='left', left_on=['nitproveedor'], right_on=['nitproveedor'])

In [ ]:
# Se carga la base de datos operadores.dta para su posterior pega
operadores = pd.read_stata('C:/Users/Miguel Angel/Documents/Supersalud/Pagos_EPS_Proveedores/operadores.dta')

In [ ]:
# Se comprueba que no haya duplicados cuando se filtra por Nitproveedor. Para esto se cuenta el número de duplicados
duplicados_operadores = operadores[operadores.duplicated(['nitproveedor'], keep='last')]
print("El número de registros duplicados es:", duplicados_operadores.nitproveedor.count(), sep='\n')

In [ ]:
# Se unen las dos bases de datos para obtener la información de FT005 con webgironoupc (Giro Directo - No_UPC - Compra de Cartera) y prestadores
FT004= pd.merge(FT004, operadores, how='left', left_on=['nitproveedor'], right_on=['nitproveedor'])

In [ ]:
# Se filtra el dataframe con las observaciones que cumplen las condiciones
FT004_V2 = FT004[
    (FT004['nitproveedor'] == '800015826')
    ]

In [ ]:
# Se crea la columna noID tomando como referencia la columna clpr_nombre
FT004['noID'] = FT004['clpr_nombre'] 

In [ ]:
# Se establecen las condiciones para establecer correctamente el noID

FT004.loc[(FT004['nit_eps'] == FT004['nitproveedor']), # Se reemplaza por EPS todo aquel proveedor que sea una EPS
       'noID'] = 'EPS'

FT004.loc[(FT004['nitproveedor'] == '"901037916"') | # Se reemplazan los NIT de la nación
       (FT004['nitproveedor'] == "900047282") |
       (FT004['nitproveedor'] == "899999014") |
       (FT004['nitproveedor'] == "900474727") |
       (FT004['nitproveedor'] == "900462447"),  
       'noID'] = 'Nacion' 

FT004.loc[FT004['noID'].isnull(), 'noID'] = 'Otro' # Para rellenar los vacios con "Otro"

FT004.loc[(FT004['clpr_nombre'] == 'IPS') & # Para corregir el IPS cuando Tipo es vacío, por IPS-OL-DM-GIM
       (FT004['tipo'].notnull()), 
       'noID'] = 'IPS-OL-DM-GIM'

FT004.loc[(FT004['tipo'].notnull()) & # Para dejar el Tipo cuando se tiene OSD u Otro
       ((FT004['noID'] == 'OSD') |
       (FT004['noID'] == 'Otro')), 
       'noID'] = FT004['tipo']

FT004.loc[(FT004['noID'] == 'Otro') & # Para cambiar el noID por Persona Natural si en Tipo Proveedor se tiene CC y CE
       ((FT004['tipoidacreedor'] == 'CC') |
       (FT004['tipoidacreedor'] == 'CE')), 
       'noID'] = 'Persona Natural'

FT004.loc[(FT004['departamento'].isnull()) & # Para asignar "Otro" al departamento si está vacío y si es una IPS o IPS Pública
       ((FT004['noID'] == 'IPS Pública') |
       (FT004['noID'] == 'IPS')), 
       'departamento'] = 'Otro'

In [ ]:
# Se convierte la variable en mes y año
FT004[['ano', 'mes']] = FT004[['ano', 'mes']].astype(float).astype(int)

In [ ]:
FT004.columns

In [ ]:
# Se agrega la información por NIT_EPS, Nitproveedor, noID, Departamento, Ano, Mes para las variables VPagado, VCausado, PW, Giro, No_UPC, Compra_Cartera
FT004 = FT004.groupby(['nit_eps', 'nitproveedor', 'noID','mes', 'ano']).agg({'CXP':'sum', 'DCXPmes':'sum'}).reset_index()

In [ ]:
# Se filtra el dataframe con las observaciones que cumplen las condiciones
FT004_V2 = FT004[
    (FT004['nitproveedor'] == '800071599')
    ]

In [ ]:
# Para guardar la base de datos lista para ser tratada en formato Excel y CSV

# FT004_V2.to_excel(r'C:/Users/Miguel Angel/Documents/Supersalud/Pagos_EPS_Proveedores/carteraprestador_2020_2021.xlsx', index = False)
FT004.to_csv(r'C:/Users/Miguel Angel/Documents/Supersalud/Pagos_EPS_Proveedores/carteraprestador_2020_2021_V2.csv', index = False, encoding='utf-8-sig')

In [ ]:
# Se eliminan la bases que no se necesitan para liberar espacio en la memoria
del [FT004, FT004_V2, FT004_dif, operadores, prestadores]